<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
from datetime import datetime
print(f"Execution Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Execution Start Time: 2025-11-11 22:21:44


In [2]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')


Using default sagemaker_studio package from environment


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
from sagemaker_studio import Projectproj = Project()s3_shared_conn = proj.connection('default.s3_shared')s3_bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]print(f"Using S3 bucket: {s3_bucket}")

SyntaxError: invalid syntax (3218661713.py, line 1)

# Test 08: GenAI ETL using Pandas

This notebook tests the capabilities of SageMind when generating ETL workloads on Spark

In [ ]:
import pandas as pdimport numpy as npfrom datetime import datetime, timedelta# Set random seed for reproducibilitynp.random.seed(42)# Generate synthetic customer datan_customers = 5000# Customer demographicscustomer_ids = [f"CUST_{str(i).zfill(6)}" for i in range(1, n_customers + 1)]ages = np.random.normal(45, 15, n_customers).astype(int)ages = np.clip(ages, 18, 80)  # Keep ages realisticgenders = np.random.choice(['Male', 'Female', 'Other'], n_customers, p=[0.48, 0.50, 0.02])# Geographic datastates = ['CA', 'NY', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI',           'NJ', 'VA', 'WA', 'AZ', 'MA', 'TN', 'IN', 'MO', 'MD', 'WI']customer_states = np.random.choice(states, n_customers)# Account informationaccount_length_months = np.random.exponential(24, n_customers).astype(int)account_length_months = np.clip(account_length_months, 1, 120)# Service usage patternsmonthly_charges = np.random.normal(75, 25, n_customers)monthly_charges = np.clip(monthly_charges, 20, 200)total_charges = monthly_charges * account_length_months + np.random.normal(0, 50, n_customers)total_charges = np.maximum(total_charges, monthly_charges)# Service featureshas_phone_service = np.random.choice([0, 1], n_customers, p=[0.1, 0.9])has_internet_service = np.random.choice(['No', 'DSL', 'Fiber optic'], n_customers, p=[0.2, 0.4, 0.4])has_online_security = np.random.choice([0, 1], n_customers, p=[0.6, 0.4])has_online_backup = np.random.choice([0, 1], n_customers, p=[0.65, 0.35])has_device_protection = np.random.choice([0, 1], n_customers, p=[0.7, 0.3])has_tech_support = np.random.choice([0, 1], n_customers, p=[0.7, 0.3])has_streaming_tv = np.random.choice([0, 1], n_customers, p=[0.6, 0.4])has_streaming_movies = np.random.choice([0, 1], n_customers, p=[0.6, 0.4])# Contract and payment infocontract_types = ['Month-to-month', 'One year', 'Two year']contracts = np.random.choice(contract_types, n_customers, p=[0.55, 0.25, 0.20])payment_methods = ['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card']payment_method = np.random.choice(payment_methods, n_customers, p=[0.35, 0.25, 0.20, 0.20])paperless_billing = np.random.choice([0, 1], n_customers, p=[0.4, 0.6])# Customer service interactionsnum_service_calls = np.random.poisson(2, n_customers)customer_satisfaction = np.random.normal(7, 2, n_customers)customer_satisfaction = np.clip(customer_satisfaction, 1, 10)# Generate churn labels with realistic patterns# Higher churn probability for certain conditionschurn_prob = 0.1  # Base churn rate# Adjust churn probability based on featureschurn_factors = np.ones(n_customers) * churn_prob# Month-to-month contracts have higher churnchurn_factors[contracts == 'Month-to-month'] *= 3# Higher monthly charges increase churn riskchurn_factors[monthly_charges > 100] *= 1.5# More service calls increase churn riskchurn_factors[num_service_calls > 4] *= 2# Lower satisfaction increases churn riskchurn_factors[customer_satisfaction < 5] *= 2.5# New customers (< 6 months) have higher churnchurn_factors[account_length_months < 6] *= 2# Cap the maximum churn probabilitychurn_factors = np.minimum(churn_factors, 0.8)# Generate churn labelschurn = np.random.binomial(1, churn_factors, n_customers)# Create the dataframecustomers_df = pd.DataFrame({    'customer_id': customer_ids,    'age': ages,    'gender': genders,    'state': customer_states,    'account_length_months': account_length_months,    'monthly_charges': np.round(monthly_charges, 2),    'total_charges': np.round(total_charges, 2),    'has_phone_service': has_phone_service,    'internet_service': has_internet_service,    'has_online_security': has_online_security,    'has_online_backup': has_online_backup,    'has_device_protection': has_device_protection,    'has_tech_support': has_tech_support,    'has_streaming_tv': has_streaming_tv,    'has_streaming_movies': has_streaming_movies,    'contract': contracts,    'payment_method': payment_method,    'paperless_billing': paperless_billing,    'num_service_calls': num_service_calls,    'customer_satisfaction': np.round(customer_satisfaction, 1),    'churn': churn})print(f"Generated {n_customers} synthetic customer records")print(f"Churn rate: {customers_df['churn'].mean():.2%}")print("\nDataset shape:", customers_df.shape)print("\nFirst few records:")customers_df.head()

In [ ]:
import boto3# Write customers_df to S3 as CSVs3_bucket = s3_buckets3_key = "datalake/landing/customers.csv"# Write the dataframe to S3customers_df.to_csv(f's3://{s3_bucket}/{s3_key}', index=False)print(f"Successfully wrote customers to s3://{s3_bucket}/{s3_key}")print(f"Shape: {customers_df.shape}")print(f"Columns: {list(customers_df.columns)}")

In [ ]:
# Load customer data from S3import pandas as pds3_path = "s3://{s3_bucket}/datalake/landing/customers.csv"df = pd.read_csv(s3_path)print(f"Successfully loaded data from S3")print(f"Dataset shape: {df.shape}")print(f"Columns: {list(df.columns)}")print("\nFirst few records:")df.head()

In [ ]:
# Data exploration - check data quality and structureprint("=== DATA EXPLORATION ===")print(f"Dataset shape: {df.shape}")print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")print("\n=== DATA TYPES ===")print(df.dtypes)print("\n=== MISSING VALUES ===")missing_values = df.isnull().sum()missing_percentage = (missing_values / len(df)) * 100missing_summary = pd.DataFrame({    'Missing Count': missing_values,    'Missing Percentage': missing_percentage})print(missing_summary[missing_summary['Missing Count'] > 0])print("\n=== DUPLICATE RECORDS ===")duplicates = df.duplicated().sum()print(f"Number of duplicate records: {duplicates}")if duplicates > 0:    print("Sample duplicate records:")    print(df[df.duplicated()].head())print("\n=== KEY STATISTICS ===")print(f"Unique customers: {df['customer_id'].nunique()}")print(f"States distribution: {df['state'].value_counts().head()}")print(f"NY customers count: {df[df['state'] == 'NY'].shape[0]}")print(f"Customers with total charges > 1000: {df[df['total_charges'] > 1000].shape[0]}")print(f"NY customers with total charges > 1000: {df[(df['state'] == 'NY') & (df['total_charges'] > 1000)].shape[0]}")print("\n=== TOTAL CHARGES DISTRIBUTION ===")print(df['total_charges'].describe())

In [ ]:
# Data cleaning - even though no nulls/duplicates, let's validate and clean if neededprint("=== DATA CLEANING ===")print(f"Original dataset shape: {df.shape}")# Create a copy for cleaningdf_clean = df.copy()# Data validation and cleaning checksprint("\n=== DATA VALIDATION ===")# Check for any potential data quality issuesprint("Checking for potential data quality issues...")# Check for negative values where they shouldn't existnegative_charges = df_clean[(df_clean['monthly_charges'] < 0) | (df_clean['total_charges'] < 0)]print(f"Records with negative charges: {len(negative_charges)}")# Check for unrealistic agesunrealistic_ages = df_clean[(df_clean['age'] < 18) | (df_clean['age'] > 120)]print(f"Records with unrealistic ages: {len(unrealistic_ages)}")# Check customer satisfaction range (should be 1-10)invalid_satisfaction = df_clean[(df_clean['customer_satisfaction'] < 1) | (df_clean['customer_satisfaction'] > 10)]print(f"Records with invalid customer satisfaction scores: {len(invalid_satisfaction)}")# Since we have no nulls or duplicates to remove, let's verify data consistencyprint(f"\n=== DATA CONSISTENCY CHECKS ===")print(f"All customer IDs are unique: {df_clean['customer_id'].nunique() == len(df_clean)}")print(f"All ages are within reasonable range: {df_clean['age'].min()} - {df_clean['age'].max()}")print(f"Customer satisfaction range: {df_clean['customer_satisfaction'].min()} - {df_clean['customer_satisfaction'].max()}")print(f"\n=== SUMMARY ===")print(f"✓ No missing values found")print(f"✓ No duplicate records found") print(f"✓ Data types are appropriate")print(f"✓ Value ranges are valid")print(f"Final cleaned dataset shape: {df_clean.shape}")df_clean.head()

In [ ]:
# Data filtering - exclude NY customers with total charges > 1000print("=== DATA FILTERING ===")print(f"Original dataset shape: {df_clean.shape}")# Before filtering - show the data we're about to filter outny_high_charges = df_clean[(df_clean['state'] == 'NY') & (df_clean['total_charges'] > 1000)]print(f"\nCustomers to be filtered out (NY with total charges > 1000): {len(ny_high_charges)}")if len(ny_high_charges) > 0:    print("Sample of records being filtered out:")    print(ny_high_charges[['customer_id', 'state', 'total_charges']].head())# Apply the filtering - EXCLUDE NY customers with total charges > 1000df_filtered = df_clean[~((df_clean['state'] == 'NY') & (df_clean['total_charges'] > 1000))]print(f"\n=== FILTERING RESULTS ===")print(f"Records before filtering: {len(df_clean)}")print(f"Records after filtering: {len(df_filtered)}")print(f"Records removed: {len(df_clean) - len(df_filtered)}")# Verify the filtering worked correctlyremaining_ny_customers = df_filtered[df_filtered['state'] == 'NY']high_charge_ny = remaining_ny_customers[remaining_ny_customers['total_charges'] > 1000]print(f"\n=== VALIDATION ===")print(f"Remaining NY customers: {len(remaining_ny_customers)}")print(f"NY customers with charges > 1000 (should be 0): {len(high_charge_ny)}")print(f"\n=== FINAL DATASET SUMMARY ===")print(f"Final dataset shape: {df_filtered.shape}")print(f"States remaining in dataset: {sorted(df_filtered['state'].unique())}")print(f"NY customers remaining (with charges <= 1000): {len(df_filtered[df_filtered['state'] == 'NY'])}")df_filtered.head()

In [ ]:
# Data Output - Save processed data to S3print("=== DATA OUTPUT ===")# Define the output path in the processed folderprocessed_s3_key = "datalake/processed/customers_processed.csv"processed_s3_path = f"s3://{s3_bucket}/{processed_s3_key}"print(f"Saving processed data to: {processed_s3_path}")# Save the filtered dataframe to S3df_filtered.to_csv(processed_s3_path, index=False)print(f"✓ Successfully saved {len(df_filtered)} records to S3")# Verify the file was created by checking S3print(f"\n=== VERIFICATION ===")print(f"Checking if file exists in S3...")try:    # List objects in the processed folder to verify    response = boto3.client('s3').list_objects_v2(        Bucket=s3_bucket,        Prefix="datalake/processed/"    )        if 'Contents' in response:        for obj in response['Contents']:            file_size_mb = obj['Size'] / (1024 * 1024)            print(f"✓ File: {obj['Key']}")            print(f"  Size: {file_size_mb:.2f} MB")            print(f"  Last Modified: {obj['LastModified']}")    else:        print("No files found in processed folder")        except Exception as e:    print(f"Error checking S3: {str(e)}")# Final ETL summaryprint(f"\n=== ETL PIPELINE SUMMARY ===")print(f"✓ Data Discovery: Located customer data in S3")print(f"✓ Data Loading: Loaded {len(df)} records from landing zone")print(f"✓ Data Exploration: Analyzed data structure and quality") print(f"✓ Data Cleaning: Validated data integrity (no issues found)")print(f"✓ Data Filtering: Removed {len(df) - len(df_filtered)} NY customers with charges > $1000")print(f"✓ Data Output: Saved {len(df_filtered)} processed records to S3")print(f"\nETL Pipeline completed successfully!")